In [2]:
import numpy as np
import pandas as pd

In [3]:
## Função para preparação do holdout (separando classes e dados) ##
def separation_features(dataframe):
    df_features = dataframe.iloc[:, :-1]
    df_classe = dataframe['classe']
    return df_features, df_classe;

## Lendo as bases: 
df1 = pd.read_csv('../Dados/CNN_Features/cnn_VGG16_256_max.csv')
df2 = pd.read_csv('../Dados/CNN_Features/cnn_VGG16_256_avg.csv')
df3 = pd.read_csv('../Dados/CNN_Features/cnn_VGG19_128_max.csv')
df4 = pd.read_csv('../Dados/CNN_Features/cnn_VGG19_256_max.csv')
df5 = pd.read_csv('../Dados/CNN_Features/cnn_VGG19_128_avg.csv')
df6 = pd.read_csv('../Dados/CNN_Features/cnn_VGG19_256_avg.csv')

df7 = pd.read_csv('../Dados/PCA_Features/PCA_cnn_VGG16_256_max.csv')
df8 = pd.read_csv('../Dados/PCA_Features/PCA_cnn_VGG16_256_avg.csv')
df9 = pd.read_csv('../Dados/PCA_Features/PCA_cnn_VGG19_128_max.csv')
df10 = pd.read_csv('../Dados/PCA_Features/PCA_cnn_VGG19_256_max.csv')
df11 = pd.read_csv('../Dados/PCA_Features/PCA_cnn_VGG19_128_avg.csv')
df12 = pd.read_csv('../Dados/PCA_Features/PCA_cnn_VGG19_256_avg.csv')


X1, y1 = separation_features(df1)
X2, y2 = separation_features(df2)
X3, y3 = separation_features(df3)
X4, y4 = separation_features(df4)
X5, y5 = separation_features(df5)
X6, y6 = separation_features(df6)
X7, y7 = separation_features(df7)
X8, y8 = separation_features(df8)
X9, y9 = separation_features(df9)
X10, y10 = separation_features(df10)
X11, y11 = separation_features(df11)
X12, y12 = separation_features(df12)

In [4]:
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [5]:
from sklearn.model_selection import train_test_split

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3,shuffle=True, random_state=1)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3,shuffle=True, random_state=1)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.3,shuffle=True, random_state=1)
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, test_size=0.3,shuffle=True, random_state=1)
X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y5, test_size=0.3,shuffle=True, random_state=1)
X6_train, X6_test, y6_train, y6_test = train_test_split(X6, y6, test_size=0.3,shuffle=True, random_state=1)
X7_train, X7_test, y7_train, y7_test = train_test_split(X7, y7, test_size=0.3,shuffle=True, random_state=1)
X8_train, X8_test, y8_train, y8_test = train_test_split(X8, y8, test_size=0.3,shuffle=True, random_state=1)
X9_train, X9_test, y9_train, y9_test = train_test_split(X9, y9, test_size=0.3,shuffle=True, random_state=1)
X10_train, X10_test, y10_train, y10_test = train_test_split(X10, y10, test_size=0.3,shuffle=True, random_state=1)
X11_train, X11_test, y11_train, y11_test = train_test_split(X11, y11, test_size=0.3,shuffle=True, random_state=1)
X12_train, X12_test, y12_train, y12_test = train_test_split(X12, y12, test_size=0.3,shuffle=True, random_state=1)


In [6]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np

#classificadores base
base_classifiers = [
    ('knn', KNeighborsClassifier()),
]

def gerar_classificadores(n):
    classificadores = []
    for i in range(n):
        nome, modelo = base_classifiers[i % len(base_classifiers)]
        classificadores.append((f'{nome}_{i}', modelo))
    return classificadores

datasets_kfold = [
    ('cnn_VGG16_256_max', X1, y1),
    ('cnn_VGG16_256_avg', X2, y2),
    ('cnn_VGG19_128_max', X3, y3),
    ('cnn_VGG19_256_max', X4, y4),
    ('cnn_VGG19_128_avg', X5, y5),
    ('cnn_VGG19_256_avg', X6, y6),
    ('PCA_cnn_VGG16_256_max', X7, y7),
    ('PCA_cnn_VGG16_256_avg', X8, y8),
    ('PCA_cnn_VGG19_128_max', X9, y9),
    ('PCA_cnn_VGG19_256_max', X10, y10),
    ('PCA_cnn_VGG19_128_avg', X11, y11),
    ('PCA_cnn_VGG19_256_avg', X12, y12),
]

tabela_final = []

for nome_base, X, y in datasets_kfold:
    print(f"\n--- Base: {nome_base} ---")
    for n in [5, 10, 15, 20]:
        estimadores = gerar_classificadores(n)
        clf = StackingClassifier(
            estimators=estimadores,
            final_estimator=LogisticRegression(),
            cv=5,
            n_jobs=-1
        )
        scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
        media = np.round(scores.mean() * 100, 2)
        print(f"{n} classificadores => {media}%")

        # Armazena os resultados para DataFrame
        tabela_final.append({
            'Base': nome_base,
            'Classificadores': n,
            'Acurácia (%)': media
        })

# Exibe como DataFrame final
df_resultados = pd.DataFrame(tabela_final)
print(df_resultados)



--- Base: cnn_VGG16_256_max ---
5 classificadores => 98.11%
10 classificadores => 98.11%
15 classificadores => 98.11%
20 classificadores => 98.24%

--- Base: cnn_VGG16_256_avg ---
5 classificadores => 98.49%
10 classificadores => 98.74%
15 classificadores => 98.74%
20 classificadores => 98.74%

--- Base: cnn_VGG19_128_max ---
5 classificadores => 93.84%
10 classificadores => 94.09%
15 classificadores => 94.21%
20 classificadores => 94.21%

--- Base: cnn_VGG19_256_max ---
5 classificadores => 98.36%
10 classificadores => 98.24%
15 classificadores => 98.24%
20 classificadores => 98.24%

--- Base: cnn_VGG19_128_avg ---
5 classificadores => 93.08%
10 classificadores => 93.33%
15 classificadores => 93.33%
20 classificadores => 93.33%

--- Base: cnn_VGG19_256_avg ---
5 classificadores => 98.24%
10 classificadores => 98.24%
15 classificadores => 98.24%
20 classificadores => 98.11%

--- Base: PCA_cnn_VGG16_256_max ---
5 classificadores => 98.99%
10 classificadores => 98.99%
15 classificadores